In [400]:
import numpy as np
import pandas as pd
import json
import re
import ast
from pandas import json_normalize

In [401]:
df = pd.read_csv(r'C:\Users\gasty\Desktop\Data\Dataset\movies_dataset.csv')
df_credits = pd.read_csv(r'C:\Users\gasty\Desktop\Data\Dataset\credits.csv')

C:\Users\gasty\AppData\Local\Temp\ipykernel_15456\3832524643.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\gasty\Desktop\Data\Dataset\movies_dataset.csv')


In [402]:
df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45

Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [403]:
df = df.drop(['adult', 'imdb_id', 'video', 'original_title', 'homepage', 'poster_path'], axis=1)

Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [404]:
# combercion a valores numericos
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')

In [405]:
# Rellenar los datos nulos con 0 solo en las columnas seleccionadas
columnas_rellenar = ['revenue', 'budget']
df[columnas_rellenar] = df[columnas_rellenar].fillna(0)

Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [406]:
df['return'] = np.where((df['revenue'] == 0) | (df['budget'] == 0), 0, df['revenue'] / df['budget'])

Los valores nulos del campo release date deben eliminarse.
las fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [407]:
df = df.dropna(subset=['release_date'])

In [408]:
error_rows = df[~df['release_date'].fillna('').astype(str).str.match(r'^\d{4}-\d{2}-\d{2}$')]
print(error_rows['release_date'])


19730     1
29503    12
35587    22
Name: release_date, dtype: object


In [409]:
df = df[~df['release_date'].isin(['1', '12', '22'])]

In [410]:
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d')

In [411]:
df['release_date'].dtype

dtype('<M8[ns]')

In [412]:
df['release_year'] = df['release_date'].dt.year

DESANIDAR
Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

In [413]:
def parse_dict_string(value):
    if pd.isna(value):
        return None
    else:
        return ast.literal_eval(value)

In [414]:
columnas_desa = ['belongs_to_collection', 'genres', 'production_companies', 'production_countries','spoken_languages']

In [415]:
for columna in columnas_desa:
    df[columna] = df[columna].apply(parse_dict_string)

In [416]:
df_belongs_to_collection = pd.json_normalize(df['belongs_to_collection']).fillna('')
df_genres = pd.json_normalize(df['genres']).fillna('')
df_production_companies = pd.json_normalize(df['production_companies']).fillna('')
df_production_countries = pd.json_normalize(df['production_countries']).fillna('')
df_spoken_languages = pd.json_normalize(df['spoken_languages']).fillna('')

In [417]:
def unir_valores(fila, key, key_aux=None):
    ids = []
    names = []
    aux = []

    for dic in fila:
        if isinstance(dic, dict):
            ids.append(str(dic.get(key, '')))
            names.append(dic.get('name', ''))
            if key_aux is not None and key_aux in dic:
                aux.append(str(dic.get(key_aux, '')))
    return ','.join(ids), ','.join(names), ','.join(aux)

belongs_to_collection

In [418]:
df_belongs_to_collection

,id,name,poster_path,backdrop_path
0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,,,,
2,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
3,,,,
4,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg
...,...,...,...,...
45371,,,,
45372,,,,
45373,,,,
45374,,,,


genres

In [419]:
df_genres

,0,1,2,3,4,5,6,7
0,"{'id': 16, 'name': 'Animation'}","{'id': 35, 'name': 'Comedy'}","{'id': 10751, 'name': 'Family'}",,,,,
1,"{'id': 12, 'name': 'Adventure'}","{'id': 14, 'name': 'Fantasy'}","{'id': 10751, 'name': 'Family'}",,,,,
2,"{'id': 10749, 'name': 'Romance'}","{'id': 35, 'name': 'Comedy'}",,,,,,
3,"{'id': 35, 'name': 'Comedy'}","{'id': 18, 'name': 'Drama'}","{'id': 10749, 'name': 'Romance'}",,,,,
4,"{'id': 35, 'name': 'Comedy'}",,,,,,,
...,...,...,...,...,...,...,...,...
45371,"{'id': 18, 'name': 'Drama'}","{'id': 28, 'name': 'Action'}","{'id': 10749, 'name': 'Romance'}",,,,,
45372,"{'id': 18, 'name': 'Drama'}",,,,,,,
45373,"{'id': 28, 'name': 'Action'}","{'id': 18, 'name': 'Drama'}","{'id': 53, 'name': 'Thriller'}",,,,,
45374,,,,,,,,


In [420]:
# Aplicar la función a cada fila del DataFrame y crear nuevas columnas "id" y "name"
df_genres[['Id', 'name', 'aux']] = df_genres.apply(lambda row: unir_valores(row, 'id'), axis=1, result_type='expand')
DF_Genres = df_genres[['Id', 'name']]

In [421]:
DF_Genres

,Id,name
0,"16,35,10751","Animation,Comedy,Family"
1,"12,14,10751","Adventure,Fantasy,Family"
2,"10749,35","Romance,Comedy"
3,"35,18,10749","Comedy,Drama,Romance"
4,35,Comedy
...,...,...
45371,"18,28,10749","Drama,Action,Romance"
45372,18,Drama
45373,"28,18,53","Action,Drama,Thriller"
45374,,


production_companies

In [422]:
df_production_companies[['Id', 'name', 'aux']] = df_production_companies.apply(lambda row: unir_valores(row, 'id'), axis=1, result_type='expand')
DF_Production_Companies = df_production_companies[['Id', 'name']]

In [423]:
DF_Production_Companies

,Id,name
0,3,Pixar Animation Studios
1,"559,2550,10201","TriStar Pictures,Teitler Film,Interscope Commu..."
2,"6194,19464","Warner Bros.,Lancaster Gate"
3,306,Twentieth Century Fox Film Corporation
4,"5842,9195","Sandollar Productions,Touchstone Pictures"
...,...,...
45371,"7025,10163,16323,38978","Westdeutscher Rundfunk (WDR),Working Title Fil..."
45372,19653,Sine Olivia
45373,6165,American World Pictures
45374,88753,Yermoliev


production_countries

In [424]:
df_production_countries[['Id', 'name', 'aux']] = df_production_countries.apply(lambda row: unir_valores(row, 'iso_3166_1'), axis=1, result_type='expand')
DF_Production_Countries = df_production_countries[['Id', 'name']]

In [378]:
DF_Production_Countries

,Id,name
0,US,United States of America
1,US,United States of America
2,US,United States of America
3,US,United States of America
4,US,United States of America
...,...,...
45371,"CA,DE,GB,US","Canada,Germany,United Kingdom,United States of..."
45372,PH,Philippines
45373,US,United States of America
45374,RU,Russia


spoken_languages

In [425]:
df_spoken_languages[['Id', 'name', 'aux']] = df_spoken_languages.apply(lambda row: unir_valores(row, 'iso_639_1'), axis=1, result_type='expand')
DF_Spoken_Languages = df_spoken_languages[['Id', 'name']]

In [380]:
DF_Spoken_Languages

,Id,name
0,en,English
1,"en,fr","English,Français"
2,en,English
3,en,English
4,en,English
...,...,...
45371,en,English
45372,tl,
45373,en,English
45374,,


Transformacion en credits

In [426]:
def extraer_directores(fila):
    directores = []
    for dic in fila:
        if isinstance(dic, dict) and dic.get('job') == 'Director':
            directores.append(dic.get('name', ''))
    return ', '.join(directores)

In [427]:
df_credits['cast'] = df_credits['cast'].apply(parse_dict_string)
df_cast = pd.json_normalize(df_credits['cast']).fillna('')

In [428]:
df_credits['crew'] = df_credits['crew'].apply(parse_dict_string)
df_crew = pd.json_normalize(df_credits['crew']).fillna('')

In [429]:
df_credits['directores'] = df_credits['crew'].apply(extraer_directores)

In [430]:
df_cast[['id', 'name', 'character']] = df_cast.apply(lambda row: unir_valores(row, 'id', 'character'), axis=1, result_type='expand')
DF_cast = df_cast[['id', 'name', 'character']]

C:\Users\gasty\AppData\Local\Temp\ipykernel_15456\2444011566.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_cast[['id', 'name', 'character']] = df_cast.apply(lambda row: unir_valores(row, 'id', 'character'), axis=1, result_type='expand')
C:\Users\gasty\AppData\Local\Temp\ipykernel_15456\2444011566.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_cast[['id', 'name', 'character']] = df_cast.apply(lambda row: unir_valores(row, 'id', 'character'), axis=1, result_type='expand')
C:\Users\gasty\AppData\Local\Temp\ipykerne

In [431]:
df_crew[['id', 'name', 'job']] = df_crew.apply(lambda row: unir_valores(row, 'id', 'job'), axis=1, result_type='expand')
Df_Crew = df_crew[['id', 'name', 'job']]

C:\Users\gasty\AppData\Local\Temp\ipykernel_15456\3120580182.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_crew[['id', 'name', 'job']] = df_crew.apply(lambda row: unir_valores(row, 'id', 'job'), axis=1, result_type='expand')
C:\Users\gasty\AppData\Local\Temp\ipykernel_15456\3120580182.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_crew[['id', 'name', 'job']] = df_crew.apply(lambda row: unir_valores(row, 'id', 'job'), axis=1, result_type='expand')
C:\Users\gasty\AppData\Local\Temp\ipykernel_15456\3120580182.py:1:

In [432]:
columnas_a_eliminar = ['cast', 'crew']
df_credits.drop(columnas_a_eliminar, axis=1, inplace=True)

cambio valores en columnas

In [433]:
df['belongs_to_collection'] = df_belongs_to_collection['name']
df['genres'] = DF_Genres['name']
df['production_companies'] = DF_Production_Companies['name']
df['production_countries'] = DF_Production_Countries['name']
df['spoken_languages'] = DF_Spoken_Languages['name']

In [434]:
df_credits['name_crew'] = df_crew['name']
df_credits['name_cast'] = df_cast['name']

In [435]:
df_credits = df_credits.reset_index(drop=True)

In [391]:
df_credits

,id,directores,name_crew,name_cast
0,862,John Lasseter,"John Lasseter,Joss Whedon,Andrew Stanton,Joel ...","Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wal..."
1,8844,Joe Johnston,"Larry J. Franco,Jonathan Hensleigh,James Horne...","Robin Williams,Jonathan Hyde,Kirsten Dunst,Bra..."
2,15602,Howard Deutch,"Howard Deutch,Mark Steven Johnson,Mark Steven ...","Walter Matthau,Jack Lemmon,Ann-Margret,Sophia ..."
3,31357,Forest Whitaker,"Forest Whitaker,Ronald Bass,Ronald Bass,Ezra S...","Whitney Houston,Angela Bassett,Loretta Devine,..."
4,11862,Charles Shyer,"Alan Silvestri,Elliot Davis,Nancy Meyers,Nancy...","Steve Martin,Diane Keaton,Martin Short,Kimberl..."
...,...,...,...,...
45471,439050,Hamid Nematollah,"Hamid Nematollah,Hamid Nematollah,Farshad Moha...","Leila Hatami,Kourosh Tahami,Elham Korda"
45472,111109,Lav Diaz,"Lav Diaz,Lav Diaz,Dante Perez,Lav Diaz,Lav Dia...","Angel Aquino,Perry Dizon,Hazel Orencio,Joel To..."
45473,67758,Mark L. Lester,"Mark L. Lester,C. Courtney Joyner,Jeffrey Gold...","Erika Eleniak,Adam Baldwin,Julie du Page,James..."
45474,227506,Yakov Protazanov,"Yakov Protazanov,Joseph N. Ermolieff","Iwan Mosschuchin,Nathalie Lissenko,Pavel Pavlo..."


In [436]:
df = df.reset_index(drop=True)

In [393]:
df

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year
0,Toy Story Collection,30000000,"Animation,Comedy,Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,7.7,5415.0,1995
1,,65000000,"Adventure,Fantasy,Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures,Teitler Film,Interscope Commu...",United States of America,1995-12-15,262797249.0,104.0,"English,Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995
2,Grumpy Old Men Collection,0,"Romance,Comedy",15602,en,A family wedding reignites the ancient feud be...,11.7129,"Warner Bros.,Lancaster Gate",United States of America,1995-12-22,0.0,101.0,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995
3,,16000000,"Comedy,Drama,Romance",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156.0,127.0,English,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995
4,Father of the Bride Collection,0,Comedy,11862,en,Just when George Banks has recovered from his ...,8.387519,"Sandollar Productions,Touchstone Pictures",United States of America,1995-02-10,76578911.0,106.0,English,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45371,NaN,0,NaN,30840,en,"Yet another version of the classic epic, with ...",5.683753,NaN,NaN,1991-05-13,0.0,104.0,NaN,Released,NaN,Robin Hood,5.7,26.0,1991
45372,NaN,0,NaN,111109,tl,An artist struggles to finish his work while a...,0.178241,NaN,NaN,2011-11-17,0.0,360.0,NaN,Released,NaN,Century of Birthing,9.0,3.0,2011
45373,NaN,0,NaN,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,NaN,NaN,2003-08-01,0.0,90.0,NaN,Released,A deadly game of wits.,Betrayal,3.8,6.0,2003
45374,NaN,0,NaN,227506,en,"In a small town live two brothers, one a minis...",0.003503,NaN,NaN,1917-10-21,0.0,87.0,NaN,Released,NaN,Satan Triumphant,0.0,0.0,1917


In [437]:
df['id'] = df['id'].astype(int)
df_credits['id'] = df_credits['id'].astype(int)

In [438]:
# Realizar la fusión de los DataFrames utilizando la columna "id" como clave
df_merged = pd.merge(df, df_credits, on='id')

In [439]:
df_merged

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return,release_year,directores,name_crew,name_cast
0,Toy Story Collection,30000000.0,"Animation,Comedy,Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,7.7,5415.0,12.451801,1995,John Lasseter,"John Lasseter,Joss Whedon,Andrew Stanton,Joel ...","Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wal..."
1,,65000000.0,"Adventure,Fantasy,Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures,Teitler Film,Interscope Commu...",United States of America,1995-12-15,262797249.0,104.0,"English,Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,4.043035,1995,Joe Johnston,"Larry J. Franco,Jonathan Hensleigh,James Horne...","Robin Williams,Jonathan Hyde,Kirsten Dunst,Bra..."
2,Grumpy Old Men Collection,0.0,"Romance,Comedy",15602,en,A family wedding reignites the ancient feud be...,11.7129,"Warner Bros.,Lancaster Gate",United States of America,1995-12-22,0.0,101.0,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,0.000000,1995,Howard Deutch,"Howard Deutch,Mark Steven Johnson,Mark Steven ...","Walter Matthau,Jack Lemmon,Ann-Margret,Sophia ..."
3,,16000000.0,"Comedy,Drama,Romance",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156.0,127.0,English,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,5.090760,1995,Forest Whitaker,"Forest Whitaker,Ronald Bass,Ronald Bass,Ezra S...","Whitney Houston,Angela Bassett,Loretta Devine,..."
4,Father of the Bride Collection,0.0,Comedy,11862,en,Just when George Banks has recovered from his ...,8.387519,"Sandollar Productions,Touchstone Pictures",United States of America,1995-02-10,76578911.0,106.0,English,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,0.000000,1995,Charles Shyer,"Alan Silvestri,Elliot Davis,Nancy Meyers,Nancy...","Steve Martin,Diane Keaton,Martin Short,Kimberl..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45446,NaN,0.0,NaN,30840,en,"Yet another version of the classic epic, with ...",5.683753,NaN,NaN,1991-05-13,0.0,104.0,NaN,Released,NaN,Robin Hood,5.7,26.0,0.000000,1991,John Irvin,"John Irvin,Sam Resnick,John McGrath,Sam Resnic...","Patrick Bergin,Uma Thurman,David Morrissey,Jür..."
45447,NaN,0.0,NaN,111109,tl,An artist struggles to finish his work while a...,0.178241,NaN,NaN,2011-11-17,0.0,360.0,NaN,Released,NaN,Century of Birthing,9.0,3.0,0.000000,2011,Lav Diaz,"Lav Diaz,Lav Diaz,Dante Perez,Lav Diaz,Lav Dia...","Angel Aquino,Perry Dizon,Hazel Orencio,Joel To..."
45448,NaN,0.0,NaN,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,NaN,NaN,2003-08-01,0.0,90.0,NaN,Released,A deadly game of wits.,Betrayal,3.8,6.0,0.000000,2003,Mark L. Lester,"Mark L. Lester,C. Courtney Joyner,Jeffrey Gold...","Erika Eleniak,Adam Baldwin,Julie du Page,James..."
45449,NaN,0.0,NaN,227506,en,"In a small town live two brothers, one a minis...",0.003503,NaN,NaN,1917-10-21,0.0,87.0,NaN,Released,NaN,Satan Triumphant,0.0,0.0,0.000000,1917,Yakov Protazanov,"Yakov Protazanov,Joseph N. Ermolieff","Iwan Mosschuchin,Nathalie Lissenko,Pavel Pavlo..."


In [396]:
#df_merged.to_csv('datos.csv', index=False)

In [440]:
df['revenue'][2997]

0.0

In [441]:
aux = df[df['belongs_to_collection'] == 'Toy Story Collection']
aux

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return,release_year
0,Toy Story Collection,30000000.0,"Animation,Comedy,Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,7.7,5415.0,12.451801,1995
2993,Toy Story Collection,0.0,"Animation,Comedy,Family",34760,en,It happened in 1983. It was a rare and remarka...,0.798428,Pixar Animation Studios,United States of America,1986-09-12,442000.0,96.0,English,Released,NaN,"'night, Mother",5.8,6.0,0.000000,1986
15334,Toy Story Collection,0.0,"Animation,Family,Comedy",79372,en,"Danny, a poor northern Californian Mexican-Ame...",0.822419,"Walt Disney Pictures,Pixar Animation Studios",United States of America,1942-05-21,0.0,105.0,"English,Español",Released,"THEY'RE STRONG FOR WINE, WOMEN, AND SONG.",Tortilla Flat,5.4,5.0,0.000000,1942
